In [1]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three
import math


In [4]:
df = pd.read_csv("/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/isee_features_data/features_MDM2-p53_test_dataset_top1.tsv", sep="\t")
df.head(21)


,mutationID,ddGexp,Evdw_wt,Eelec_wt,Edesolv_wt,BSA_wt,Evdw_diff,Eelec_diff,Edesolv_diff,BSA_diff,...,F,P,S,T,W,Y,V,PSSM_wt,PSSM_diff,PSSMic
0,1YCR_A_GLN48ALA,0.46115,-61.4243,-220.279,-39.5198,1483.89,0.2114,-23.610,2.6658,29.34,...,-4,-2,-1,-2,-3,-2,-3,7,-9,0.98
1,1YCR_A_GLN48PHE,0.97618,-61.4243,-220.279,-39.5198,1483.89,-5.4529,39.624,-9.4536,41.95,...,-4,-2,-1,-2,-3,-2,-3,7,-11,0.98
2,1YCR_A_GLU1LYS,0.09590,-61.4243,-220.279,-39.5198,1483.89,0.2334,-77.192,0.9748,78.19,...,-4,-2,-1,-1,-4,-3,-3,5,-4,0.79
3,1YCR_A_HIS49ALA,0.72481,-61.4243,-220.279,-39.5198,1483.89,-5.6492,1.781,1.7714,17.18,...,-2,0,-2,-3,-4,1,-4,10,-13,1.69
4,1YCR_A_HIS49GLY,1.08229,-61.4243,-220.279,-39.5198,1483.89,-6.0753,-2.124,1.0235,52.25,...,-2,0,-2,-3,-4,1,-4,10,-13,1.69
5,1YCR_A_LEU30TRP,1.41361,-61.4243,-220.279,-39.5198,1483.89,-6.2831,-17.768,-1.1288,10.79,...,1,-3,-2,0,-2,-1,1,2,-4,0.53
6,1YCR_A_LYS27ALA,0.65342,-61.4243,-220.279,-39.5198,1483.89,0.2628,20.360,-1.2872,1.81,...,-4,0,0,-1,-4,-3,-3,6,-7,0.66
7,1YCR_A_LYS27GLU,1.19490,-61.4243,-220.279,-39.5198,1483.89,-2.7336,36.901,-0.5130,44.12,...,-4,0,0,-1,-4,-3,-3,6,-6,0.66
8,1YCR_A_LYS70GLU,2.09604,-61.4243,-220.279,-39.5198,1483.89,-10.1175,95.818,-3.8077,40.02,...,-4,-2,0,-1,-4,-3,-3,6,-6,0.77
9,1YCR_A_MET26ALA,2.09832,-61.4243,-220.279,-39.5198,1483.89,-3.3882,29.961,4.5753,39.50,...,1,-3,-2,-1,-2,1,2,6,-7,0.43


In [5]:
# Generate 4 seq score file

for r in df.values:
    pid, chain_ID, mutid = r[0].split('_')
    print(pid+'_'+'A\t' + pid+'_'+'B\t' + pid+'_'+'A\t' + r[0] + '\t' + str(r[1]))

1YCR_A	1YCR_B	1YCR_A	1YCR_A_GLN48ALA	0.46115
1YCR_A	1YCR_B	1YCR_A	1YCR_A_GLN48PHE	0.9761799999999999
1YCR_A	1YCR_B	1YCR_A	1YCR_A_GLU1LYS	0.0959
1YCR_A	1YCR_B	1YCR_A	1YCR_A_HIS49ALA	0.7248100000000001
1YCR_A	1YCR_B	1YCR_A	1YCR_A_HIS49GLY	1.08229
1YCR_A	1YCR_B	1YCR_A	1YCR_A_LEU30TRP	1.41361
1YCR_A	1YCR_B	1YCR_A	1YCR_A_LYS27ALA	0.65342
1YCR_A	1YCR_B	1YCR_A	1YCR_A_LYS27GLU	1.1949
1YCR_A	1YCR_B	1YCR_A	1YCR_A_LYS70GLU	2.0960400000000003
1YCR_A	1YCR_B	1YCR_A	1YCR_A_MET26ALA	2.09832
1YCR_A	1YCR_B	1YCR_A	1YCR_A_MET26GLY	2.09832
1YCR_A	1YCR_B	1YCR_A	1YCR_A_MET38ALA	-0.14420999999999998
1YCR_A	1YCR_B	1YCR_A	1YCR_A_THR2ALA	-0.01207
1YCR_A	1YCR_B	1YCR_A	1YCR_A_THR2ASN	0.25305
1YCR_A	1YCR_B	1YCR_A	1YCR_A_TYR76PHE	-0.19415
1YCR_A	1YCR_B	1YCR_A	1YCR_A_VAL69ALA	0.24859
1YCR_A	1YCR_B	1YCR_A	1YCR_B_ASN13ALA	0.89866
1YCR_A	1YCR_B	1YCR_A	1YCR_B_ASN13THR	-0.07287
1YCR_A	1YCR_B	1YCR_A	1YCR_B_GLU12ALA	0.9022600000000001
1YCR_A	1YCR_B	1YCR_A	1YCR_B_GLU12LYS	2.07654
1YCR_A	1YCR_B	1YCR_A	1YCR_B_GLU1ALA	1.26884
1

In [20]:
# input: pdb ID, mutation ID (ISEE form)
# output: wild, mut sequence
def get_seq_wild_mut(pid, chain_ID, mutid):
#     print(pid, chain_ID, mutid)
    
    parser = PDBParser()
    ent_name = pid.lower()

    structure = parser.get_structure(ent_name, '/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/isee_features_data/pdb/' + ent_name.lower() + '.pdb')
#     structure = parser.get_structure(ent_name, '/home/zgy_ucla_cs/Research/Protein/PPI-Binding/skempi/pdb_new/pdb' + ent_name + '.ent')

    assert(len(structure) == 1)
    model = structure[0]
    chain = model[chain_ID]
    w_seq0 = ''
    ppb = PPBuilder()
    for pp in ppb.build_peptides(chain):
    #     print('Ch0',pp.get_sequence())
        w_seq0 += pp.get_sequence()
#     print('Ch B', w_seq0)
    
    r_w = mutid[:3]
    r_m = mutid[-3:]
    residue_ID = mutid[3:-3]
    print(residue_ID, r_w, r_m)
    
#     Mutation
    mut_chain = model[chain_ID].copy()
    
    if residue_ID.isdigit():
        residue = mut_chain[int(residue_ID)]
    else:
        residue = mut_chain[(' ', int(residue_ID[:-1]), residue_ID[-1].upper())]

    print(residue.resname)
    assert(residue.resname == r_w)
    residue.resname = r_m

    m_seq0 = ''
    for pp in ppb.build_peptides(mut_chain):
    #     print('Ch0',pp.get_sequence())
        m_seq0 += pp.get_sequence()
    
    return w_seq0, m_seq0
    
    
    

In [21]:
id_seq = {}
# fout = open('/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/processed/NM_test.seq.txt', 'w')
fout = open('/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/processed/MDM2-p53_test.seq.txt', 'w')

for r in df.values:
    pid, chain_ID, mutid = r[0].split('_')
#     print(pid, chain_ID, mutid)
    w_seq, m_seq = get_seq_wild_mut(pid, chain_ID, mutid)
    wid = pid+'_'+chain_ID
    mid = pid+'_'+chain_ID+'_'+mutid
    if wid not in id_seq:
        print(wid+'\t'+w_seq, file=open('/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/processed/MDM2-p53_test.seq.txt', "a"))
        id_seq[wid] = w_seq
    if mid not in id_seq:
        print(mid+'\t'+m_seq, file=open('/home/zgy_ucla_cs/Research/Protein/PPI-Binding/iSee-master/processed/MDM2-p53_test.seq.txt', "a"))
        id_seq[mid] = m_seq
#     chain_ID, residue_ID, r_w, r_m = mutation_pdb[1], mutation_pdb[2:-1], mutation_pdb[0], mutation_pdb[-1]
#     print(chain_ID, residue_ID, r_w, r_m)
#     break

48 GLN ALA
TYR


AssertionError: 